In [43]:
# imports
import seaborn as sbn
import json, re, time, unicodedata, unidecode, codecs, random, math, warnings, itertools, os
from pprint import pprint
from lxml import etree
from pattern.nl import parse, split, parsetree
from collections import defaultdict, Counter
from datetime import date, datetime, timedelta
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from pattern.nl import sentiment
import pandas as pd
from scipy.stats.mstats import normaltest
from tqdm import tqdm_notebook as tqdm

In [42]:
tqdm.__version__

'4.19.6'

In [18]:
# files
#MWE
#topics = json.load(open('D:\\4. Data\\Amazones_Forum_Export_JSON\\MWE_topic.json'))
#posts = json.load(open('D:\\4. Data\\Amazones_Forum_Export_JSON\\MWE.json'))
#regular
forums = json.load(open('D:\\4. Data\\Amazones_Forum_Export_JSON\\2017-12-07T13-35-45 _amazones_forums_export.json'))
# zorg voor een nieuwe versie van dit bestand; verkeerd opgeslagen dus je mist het kontje!
topics = json.load(open('D:\\4. Data\\Amazones_Forum_Export_JSON\\2017-12-07T13-36-51_amazones_forum_topics_export.json'))
posts = json.load(open('D:\\4. Data\\Amazones_Forum_Export_JSON\\2017-12-07T13-39-20_amazones_forum_posts_export.json'))
users = json.load(open('D:\\4. Data\\Amazones_Forum_Export_JSON\\2017-12-07T13-39-20_amazones_users_export.json'))

In [19]:
# remove non-ascii characters from file (otherwise they will become part of the tokens)
def remove_non_ascii(text):
    return unidecode.unidecode(text)
    #return ''.join([i if ord(i) < 128 else ' ' for i in text])

In [20]:
def cleanup(text):
    #remove all links, images, quotes, and emailaddresses
    text=re.sub('<a.*?>(.*?)</a>','',text) #remove links
    text=re.sub('(http:|www)\S*','',text) #remove links without markup
    text=re.sub('\[\\\/url\]','',text)
    text=re.sub('<img.*?/>', '',text) #remove images
    text=re.sub('<div class="bb-quote">((\s|\S)*?)</div>','',text) #remove quotes
    text=re.sub('<script.*?>([\S\s]*?)</script>','',text) #remove emailaddresses

    #replace all emoticon-icons
    text=re.sub('<img.*?title="(.*?)".*?/>', '(EMO:\\1)',text) #replace emoticons by textual indicators 

    # replace (most) sideways latin emoticons
    text=re.sub('[^>]:-?(\)|\])','(EMO:smiley)',text)
    text=re.sub(u'☺️','(EMO:smiley)',text)
    text=re.sub('[^>]:-?(\(|\[)','(EMO:sad)',text)
    text=re.sub(';-?(\)|\])','(EMO:wink)',text)
    text=re.sub(r'(:|;|x|X)-?(D)+\b','(EMO:laugh)',text)
    text=re.sub(':-?(/|\\\|\|)','(EMO:frown)',text)
    text=re.sub(r'(:|;)-?(p|P)+\b','(EMO:cheeky)',text)
    text=re.sub('(:|;)(\'|\")-?(\(|\[)','(EMO:cry)',text)
    text=re.sub('\<3+','(EMO:heart)',text)
    text=re.sub(u'❤️','(EMO:heart)',text)
    text=re.sub('((\>:-?(\(|\]))|(\>?:-?@))','(EMO:angry)',text)
    text=re.sub('\>:-?(\)|\])','(EMO:evil)',text)
    text=re.sub(r'(:|;)-?(O|o|0)+\b','(EMO:shock)',text)
    text=re.sub('(:|;)-?(K|k|x|X)','(EMO:kiss)',text)
    # :s
    # :x is eigenlijk geen kus, geloof ik...


    #other important adjustments:
    text=re.sub('m\'?n\s','mijn ',text) # replacing m'n and mn with mijn, so it gets parsed correctly.
    text=re.sub('z\'?n\s','zijn ',text) #replacing z'n and zn with zijn
    text=re.sub('d\'?r\s','haar ',text) #replacing d'r and dr with zijn (only if followed by space, so dr. stays dr.)

    # replace all emoticons (and other things) written between double colons
    text=re.sub(':([a-zA-Z]+):','(EMO:\\1)',text)

    # remove remaining markup
    text=re.sub('</?(ol|style|b|p|em|u|i|strong|br|span|div|blockquote|li)(.*?)/?>','',text)
    text=re.sub('(\[|\]|\{|\})', '',text)

    # separate text from punctuation (may cause double/triple spaces - does not matter at this point)
    text = re.sub('(\.{2,}|/|\)|,|!|\?)','\\1 ',text) # space behind
    text=re.sub('(/|\()',' \\1',text) # space in front
    text=re.sub('(\w{2,})(\.|,)','\\1 \\2 ',text) #space 'between'

    return(text)

In [21]:
# make two dictionaries: user's post T imes, and user's P ost texts.
def make_P_T_and_D(topics,posts):
    P = defaultdict(list)
    T = defaultdict(list)
    D = []

    with tqdm(total=len(topics)) as pbar:
        for t in reversed(topics):
            pbar.update(1)
            P[t['Author uid']].append((remove_non_ascii(cleanup(t["Body"])),1))
            T[t['Author uid']].append(t['Post date'])
            D.append(datetime.strptime(t['Post date'], '%d/%m/%Y - %H:%M'))

    with tqdm(total=len(posts)) as pbar:
        for p in reversed(posts):
            pbar.update(1)
            P[p['Auteur-uid']].append((remove_non_ascii(cleanup(p["Body"])),0))
            T[p['Auteur-uid']].append(p['Datum van inzending'])
            D.append(datetime.strptime(p['Datum van inzending'], '%d/%m/%Y - %H:%M'))

    return (P,T,D)

In [22]:
def make_binlist(D,timetick=1): #timetick in days
    lower = min(D)
    upper = max(D)

    if lower.time()>=datetime.strptime('4:00','%H:%M').time():
        lower = lower.replace(hour = 4, minute = 0)
    else:
        lower = (lower+timedelta(days = -1)).replace(hour=4,minute=0)

    if upper.time()<datetime.strptime('12:00','%H:%M').time():
        upper = upper.replace(hour = 4, minute = 0)
    else:
        upper = (upper+timedelta(days=1)).replace(hour=4,minute=0)

    return([lower + timedelta(hours=x) for x in range(0, 24*(upper-lower).days, timetick)])

In [23]:
def determine_questionmarks(body):
    Q = 0
    for sentence in sent_tokenize(body):
        if re.search('\?+', sentence):
            Q+=1
    if len(sent_tokenize(body))!=0:
        return float(Q)/float(len(sent_tokenize(body)))
    else:
        return 0

# expects a string
# returns an int, representing the average sentiment score per sentence in the string, calculated by pattern
def determine_sentiment(body):
    S = []
    for sentence in sent_tokenize(body):
        S.append(sentiment(sentence)[0])
    return np.mean(S)

# expects a string
# returns an int, representing the average objectivity score per sentence in the string, calculated by pattern
def determine_objectivity(body):
    O = []
    for sentence in sent_tokenize(body):
        O.append(sentiment(sentence)[1])
    return np.mean(O)

# expects a string
# returns an int, representing the total nr of sentences the string is built of
def determine_length(body):
    # in sentences:
    return(len(sent_tokenize(body)))

# expects a string
# returns an int, representing the average nr of words in sentences that occur in the string
def determine_sent_length(body):
    # in words:
    L = []
    for sentence in sent_tokenize(body):
        L.append(len(word_tokenize(sentence)))
    return np.mean(L)

In [24]:
def compare_variables(pp,user,nr_of_posts,mean_quest,mean_object,mean_sents,mean_length,mean_sents_length,nr_of_starts,nr_of_responses):
    fig = plt.figure(1)
    ax = plt.subplot(111)

    posts, = ax.plot(nr_of_posts.values(),'b.', label = 'nr of posts', alpha = 0.5) #blue
    senlength, = ax.plot(mean_sents_length.values(),'k.', label = 'mean sentence length (words)',alpha = 0.5) #yellow
    postlength, = ax.plot(mean_length.values(),'c.', label = 'mean post length (sentences)',alpha = 0.5) #cyan
    starts, = ax.plot(nr_of_starts.values(),'g.', label = 'start posts', alpha = 0.5) #black
    #responses, = ax.plot(nr_of_responses.values(),'y.', label = 'response posts', alpha = 0.5)
    
    first_legend = plt.legend(handles=[posts,senlength,postlength,starts], title = "left axis", loc='upper left', bbox_to_anchor=(0, -0.1),ncol=1)
    axx = plt.gca().add_artist(first_legend)
    
    ax1 = ax.twinx()
    qmarks, = ax1.plot(mean_quest.values(),'y.', label = 'question ratio',alpha = 0.5) #green
    ovalues, = ax1.plot(mean_object.values(),'r.', label = 'subjectivity',alpha = 0.5) #red
    svalues, = ax1.plot(mean_sent.values(),'m.', label = 'sentiment',alpha = 0.5) #magenta
    
    
    second_legend = plt.legend(handles=[qmarks,ovalues,svalues], title = "right axis",loc='upper right', bbox_to_anchor=(1, -0.1),ncol=1)

    ax.set_ylim(0,40)
    ax1.set_ylim(-1,1)
    
    box = ax.get_position()
    ax.set_position([box.x0, box.y0 + box.height, box.width, box.height * 0.7])
    box = ax1.get_position()
    ax1.set_position([box.x0, box.y0 + box.height, box.width, box.height * 0.7])
    
    plt.title(user)
    plt.show()
    pp.savefig(fig, dpi = 300, transparent = True)

In [25]:
def compare_two(user,x,y):
    x = x.values()
    y = y.values()
    
    plt.plot(x,y, 'bo', alpha = 0.1)
    plt.xlabel('x-axis variable')
    plt.ylabel('y-axis variable')
    plt.xlim(0,30)
    plt.ylim(-1,1)
    plt.title(user)
    plt.show()

In [26]:
# determine which variables are non-normal, and make a mask for correlations that are measured nonparametrically
def mask_nonnormal(df,mask):
    for column in df:
        index = df.columns.get_loc(column)
        z,p = normaltest(df[column].tolist())
        # als de data in de kolom significant niet-normaal verdeeld is, activeer dan het maskeer voor de relevante rij en kolom
        if p<0.05:
            mask[index]=True #rijen blokkeren
            mask[0::1,index]=True #kolommen blokkeren
    return mask

In [27]:
def visualise(df, method):
    global averagedict
    rho = df.corr(method = m)
    mask = np.zeros_like(rho, dtype=np.bool)
    
    if method == 'pearson' or method == 'spearman':
        mask = mask_nonnormal(df,mask)
        cmap = sbn.diverging_palette(120, 260, n=21, s=80)
        cbar_kws= dict(use_gridspec=False,location="left",label= "Pearson's Rho")
    else:
        mask = ~mask_nonnormal(df,mask)
        cmap = sbn.diverging_palette(10, 30, n=21, s=99, l=65)
        cbar_kws = dict(use_gridspec=False,location="right",label= "Kendall's Tau")

    #mask the upper half of the figure
    mask[np.triu_indices_from(mask)] = True
    
    # calculate averages        
    for i,row in enumerate(mask):
        for j,column in enumerate(row):
            if column == False: # all unmasked positions
                averagedict[i,j].append(rho.iloc[i][j])
                if method == 'pearson' or method == 'spearman':
                    normaldict[i,j].append(rho.iloc[i][j])
                else:
                    nonnormaldict[i,j].append(rho.iloc[i][j])
                    countnonnormal[i,j].append(1)
    
    # make heatmap
    ax = sbn.heatmap(rho, mask=mask, cbar_kws = cbar_kws, cmap=cmap, vmin = -1, vmax = 1, annot = True, fmt='1.2f')
    plt.title('results of (non-)parametric correlation tests')
    #plt.show()

In [28]:
def show_correlations(dictionaries,countnonnormal,rho):
    for d in dictionaries:
        frame = pd.DataFrame().reindex_like(rho)
        for (i,j) in d:
            frame.iloc[i][j]=np.nanmean(d[(i,j)])
        print frame

    frame = pd.DataFrame().reindex_like(rho)
    for (i,j) in countnonnormal:
        frame.iloc[i][j]=np.nansum(countnonnormal[(i,j)])
    print frame

In [29]:
def determine_week_activity(first_date,last_date,bindict):
    for d in range(0, (last_date-first_date).days,7):
        week_start = first_date+timedelta(days=d)
        week_end = first_date+timedelta(days=d+7)
        for date in list(itertools.chain.from_iterable(bindict.values())):
            if week_start<=datetime.strptime(date, '%d/%m/%Y - %H:%M')<week_end:
                weekcountdict[week_start,week_end].append(1)
        if len(weekcountdict[week_start,week_end])==0:
            weekcountdict[week_start,week_end] = []
    return weekcountdict

In [82]:
def determine_past_activity(bindict,index,hours_back=24):
    if 0<= index-hours_back<=len(bindict):
        past_activity = np.sum([len(bindict[binlist[index-x],binlist[index+1-x]]) for x in range(hours_back)])
    else:
        past_activity=0
    return past_activity

In [36]:
def print_information(user, T, first_date,last_date,activity,activity_nonempty,weekcount):
    print "User:", user
    print "posted one or more posts in", len(T[user]), "'bins'." 
    print "The first post: ", first_date
    print "The last post: ", last_date
    print "Activity spread over: ", last_date-first_date
    print "The average nr of posts per week: ", np.mean(activity), "including long times of inactivity."
    print "The average nr of posts in non-empty weeks: ", np.mean(activity_nonempty)
    print "The range of activity: ", min([len(x) for x in weekcount.values()]), " to ", max([len(x) for x in weekcount.values()]), " posts per week"
    print

In [30]:
PTD = make_P_T_and_D(topics,posts) 
P = PTD[0]
T = PTD[1]
D = PTD[2]

A Jupyter Widget

A Jupyter Widget

In [86]:
plt.close()
path = r"C:\Users\sternheimam\Desktop\my-notebook\user-csvs" 

binlist = make_binlist(D,1) #timetick in (whole) days
pp = PdfPages("plots-author-BVN.pdf")
normaldict = defaultdict(list)
nonnormaldict = defaultdict(list)
averagedict = defaultdict(list)
countnonnormal = defaultdict(list)
over_treshold = []
neg_diff = 24

# first list all usable users (enables nice progress bar)
for user in T:
    if len(T[user])<30:
        pass # go to next user. This one has too little activity
    else:
        over_treshold.append(user)
        
with tqdm(total=len(over_treshold)) as processbar:
    for user in over_treshold:
        print user,
        processbar.update(1)
        
        first_date = 0
        last_date = 0
        
        csv_date = []
        csv_sentiment = []
        csv_questionmarks = []
        csv_subjectivity = []
        csv_sentencelength = []
        csv_postlength = []
        csv_startposts = []
        csv_inactivity = []
        csv_backtrack = []
        
        # assume an inactivity of 0 time bins
        inactivity = 0
        
        # variables
        bindict = defaultdict(list)
        postdict = defaultdict(list)
        metadict = defaultdict(list)
        weekcountdict = defaultdict(list)
        backtrackdict = defaultdict(list)
        
        # linguistic markers, like adjectives / pronouns / emoticons, and the diversity of topics / vocabulary
        # time relative posts (see email from stephan)

        # 1) look through the sorted list of datetimes
        for index,boundary in enumerate(tqdm(binlist)):
            if index+1>=len(binlist):
                break
            else:
                # 1a) set time bin boundaries
                lower = binlist[index]
                upper = binlist[index+1]
                
                #1b) put all user's posts in a dictionary (keyed by the time bin boundaries) 
                for time in T[user]:
                    #update one in tqdm
                    if lower<=datetime.strptime(time, '%d/%m/%Y - %H:%M')<upper:
                        if first_date == 0:
                            first_date = datetime.strptime(time, '%d/%m/%Y - %H:%M')
                            last_date = datetime.strptime(time, '%d/%m/%Y - %H:%M')
                        else:
                            last_date = datetime.strptime(time, '%d/%m/%Y - %H:%M')
                                                  
                        body = P[user][T[user].index(time)][0]
                        meta = P[user][T[user].index(time)][1]

                        bindict[lower,upper].append(time)
                        past_activity = determine_past_activity(bindict,index,24)
                        
                        postdict[lower,upper].append(body) 
                        metadict[lower,upper].append(meta)
                        backtrackdict[lower,upper].append(past_activity)
                        
                # fill up empty places in dictionary
                if len(bindict[lower,upper])==0:
                    bindict[lower,upper]=[]
                    postdict[lower,upper]=[]
                    metadict[lower,upper]=[]
                    backtrackdict[lower,upper]=[]

                #convert dictionaries to single values per bin, for ML purposes
                body = '. '.join(postdict[lower,upper]) #can be empty!
                csv_date.append(lower)
            
                # when no post was posted within the boundaries of this bin, import nan value and add one to the inactivity variable
                if len(body) == 0:
                    csv_sentiment.append(float('nan'))
                    csv_questionmarks.append(float('nan'))
                    csv_subjectivity.append(float('nan'))
                    csv_sentencelength.append(float('nan'))
                    csv_postlength.append(float('nan'))
                    csv_startposts.append(float('nan'))
                    inactivity+=1
                    csv_inactivity.append(float('nan'))
                    csv_backtrack.append(float('nan'))
                else:
                    csv_sentiment.append(determine_sentiment(body))
                    csv_questionmarks.append(determine_questionmarks(body))
                    csv_subjectivity.append(determine_objectivity(body))
                    csv_sentencelength.append(determine_sent_length(body))
                    csv_postlength.append(np.mean([determine_length(x) for x in postdict[lower,upper]]))
                    csv_startposts.append(np.mean(metadict[lower,upper]))
                    csv_inactivity.append(inactivity)
                    csv_backtrack.append(np.sum(backtrackdict[lower,upper][-1]))
                    inactivity = 0

        # we have collected all data for a single user, and now we do stuff with it:
        weekcount = determine_week_activity(first_date,last_date,bindict)   
        activity = [len(x) for x in weekcount.values()]
        activity_nonempty = [len(x) for x in weekcount.values() if not x==[]]
        #print_information(user, T, first_date,last_date,activity,activity_nonempty,weekcount)            
                
        # maak dataframe van alle variabelen
        df = pd.DataFrame({"Date & Time": csv_date, "Sentiment": csv_sentiment, "Questions": csv_questionmarks, 
                           "Subjectivity": csv_subjectivity, "Words/Sentence": csv_sentencelength, 
                           "Sentences/Post": csv_postlength, "First posts": csv_startposts,
                           "Inactivity": csv_inactivity, "posts in last 24H": csv_backtrack}).dropna()
        name = "features_user_"+str(user)+".csv"       
        df.to_csv(os.path.join(path,name),index=False)

A Jupyter Widget

2317

A Jupyter Widget

Exception in thread Thread-29:
Traceback (most recent call last):
  File "C:\Users\sternheimam\AppData\Local\Continuum\anaconda2\lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "c:\python27\src\tqdm\tqdm\_tqdm.py", line 148, in run
    for instance in self.tqdm_cls._instances:
  File "C:\Users\sternheimam\AppData\Local\Continuum\anaconda2\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



 2314

...

 3779

A Jupyter Widget

 2440

A Jupyter Widget

KeyboardInterrupt: 

In [53]:
1>=1

True

In [24]:
len(above30)

456

In [2]:
#determine nr of posts in a week

first_date = datetime.strptime("25/10/2014 - 09:36", '%d/%m/%Y - %H:%M')
last_date = first_date + timedelta(days=366)
dates = [first_date + timedelta(days=62),first_date + timedelta(days=23),first_date + timedelta(days=14),first_date + timedelta(days=54)]

for d in range(0,(last_date-first_date).days,7):
    border1 = first_date+timedelta(days=d)
    border2 = first_date+timedelta(days=d+7)
    for a_date in dates:
        if border1<=a_date<border2:
            print border1, " \ ", a_date, " / ", border2
            # write to dictionary with borders as keys (start dict at first activity, end at last)

# determine average length of values in dictionary

2014-11-08 09:36:00  \  2014-11-08 09:36:00  /  2014-11-15 09:36:00
2014-11-15 09:36:00  \  2014-11-17 09:36:00  /  2014-11-22 09:36:00
2014-12-13 09:36:00  \  2014-12-18 09:36:00  /  2014-12-20 09:36:00
2014-12-20 09:36:00  \  2014-12-26 09:36:00  /  2014-12-27 09:36:00
